In [1]:
import numpy as np
import pandas as pd

from urllib.request import urlopen
from bs4 import BeautifulSoup as bs

In [2]:
##target_url_science: 소분야_과학
##target_url_energy : 소분야_에너지
target_url_science ='https://www.etnews.com/news/section.html?id1=20&id2=020'
target_url_energy = 'https://www.etnews.com/news/section.html?id1=20&id2=065'

targets= [target_url_science, target_url_energy]
#1 article_url구하기
#1-1. target_url_science의 last_page보다 무조건 큰수(1000)를 가진 url을 입력. 그러면 last_page로 저절로 이동함. 그리고 last_page를 추출함.

base_url = "https://www.etnews.com"
article_url = []
for target in targets:
    url = target + '&page=1000'
    html = urlopen(url)
    soup = bs(html.read(),"html.parser")
    last_page = soup.find('a',{'href':'javascript:;'}).text

    for page in range(1,int(last_page)+1):
        url_1 = target+"&page="+str(page)
        html_1 = urlopen(url_1)
        soup = bs(html_1.read(),"html.parser")

        for links in soup.select('p > a'):
            link = links.attrs['href']
            if "https" not in link:
                article_url.append(base_url+link)

In [4]:
len(article_url)

1514

In [4]:
crawl_df = pd.DataFrame(columns=['분야','발행일','제목','내용'])

In [5]:
for link in article_url:
    html = urlopen(link)
    soup = bs(html.read(),'html.parser')
    
    category = '과학'
    
    date = soup.find_all('time', {'class':'date'})[0].text[6:16]

    title = soup.find_all('h2')[0].text
    
    draft = soup.find_all('p')

    context = ''   
    for i in draft:
        if i.a == None:
            i = i.text.replace("\n","")
            context += i
            
    tmp_df = pd.DataFrame([[category,date,title,context]],columns=crawl_df.columns)
    crawl_df = pd.concat([crawl_df, tmp_df])

In [6]:
crawl_df

,분야,발행일,제목,내용
0,과학,2023-09-13,[과학기술이 미래다] 〈102〉국내 첫 퍼스널 컴퓨터 경진대회,1984년 4월 22일. 제1회 퍼스널 컴퓨터 경진대회가 이날 오전 9시 서울 송파...
0,과학,2023-09-13,나노·소재 성과공유회,나노·소재분야 R&D 정책 및 우수 연구성과를 발표하고 성과물을 공유하는 나노·소재...
0,과학,2023-09-12,[알림] 테크비즈코리아 2023,전자신문사가 올해에도 우리 연구 현장의 사업화 유망기술을 민간기업과 투자자에 소개하...
0,과학,2023-09-12,드럼치는 로봇 '모펫' 입니다,국내 연구진이 음악 빅데이터 기반 인공지능(AI)을 통해 인간과 상호작용하며 합주 ...
0,과학,2023-09-11,드럼치는 로봇 '모펫' 입니다,국내 연구진이 음악 빅데이터 기반 인공지능(AI)을 통해 인간과 상호작용하며 합주 ...
...,...,...,...,...
0,과학,2023-03-16,"지질연, 리튬 최대 생산지 서호주와 핵심광물 협력 약속","한국지질자원연구원(원장 이평구)은 15일, 서호주 광물자원연구소(MRIWA)와 인터..."
0,과학,2023-03-16,과학영재학교 4대 과기원 조기 진학 가능해진다,이르면 2025학년도부터는 과학영재학교 학생의 4대 과학기술원 조기 진학이 가능해진...
0,과학,2023-03-16,"KAIST, AI로 코로나19 치료제 팍스로비드와 기존 약물 반응 예측 고도화",한국과학기술원(KAIST·총장 이광형)은 이상엽 생명화학공학과 특훈교수팀이 인공지능...
0,과학,2023-03-16,"ETRI, 지난해 국제표준특허 60건 추가…총 1077건 보유",한국전자통신연구원(ETRI)이 지속된 노력으로 국제표준화 부문에서 큰 성과를 거두고...


In [ ]:
crawl_df = pd.DataFrame(columns=['분야','발행일','제목','내용','전기(빈도수)',\
                                 '에너지(빈도수)', '원자력(빈도수)', '수력(빈도수)', '화력(빈도수)'])

In [ ]:
# 1-3. 기사의 '분야','발행일','제목','내용','전기(빈도수)',
#      '에너지(빈도수)', '원자력(빈도수)', '수력(빈도수)', '화력(빈도수)' 정보를 추출해보자.